In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import re
from tqdm.auto import tqdm
tqdm.pandas()

In [2]:
df = pd.read_csv('extracted_csv01.csv')

In [3]:
df.sample(5)

,src,tgt
94449,Churchill spokesman John Asher said without th...,"If the larger cut had not been made , Churchil..."
139469,Political observers here are quick to point ou...,"Nevada voters , like those in other important ..."
99287,The discovery of the limestone tombstone and s...,"While digging up a car park in Cirencester , a..."
142850,"Within seconds , he had swallowed both water a...",Consuming both water and fish in a matter of s...
70111,"However , its funding was cancelled and only t...","Its funding was terminated , resulting in only..."


In [4]:
df['src_char_count'] = df['src'].astype('str').apply(lambda x:len(x))
df['tgt_char_count'] = df['tgt'].astype('str').apply(lambda x:len(x))

In [5]:
df['src_word_count'] = df['src'].astype('str').apply(lambda x:len(x.split()))
df['tgt_word_count'] = df['tgt'].astype('str').apply(lambda x:len(x.split()))
df

,src,tgt,src_char_count,tgt_char_count,src_word_count,tgt_word_count
0,"Upon its arrival in Canberra , the Olympic fla...",Chinese officials handed over the Olympic flam...,148,143,26,25
1,"She , in turn , offered them a message stick ,...","As a gesture of peace and welcome , she presen...",79,77,19,16
2,Hundreds of pro-Tibet protesters and thousands...,"According to reports , hundreds of demonstrato...",88,114,12,17
3,Demonstrators and counter-demonstrators were k...,The Australian Federal Police prevented demons...,90,106,12,13
4,Preparations for the event were marred by a di...,A dispute between Australian and Chinese offic...,224,209,35,33
...,...,...,...,...,...,...
172054,He wants to see prices brought down across the...,He desires a reduction in prices across all ca...,303,301,53,48
172055,"Costs: The Department for Environment , Food a...","The Department for Environment , Food and Rura...",191,197,36,33
172056,Details of the action on water bills are expec...,Downing Street is expected to reveal specifics...,169,160,34,30
172057,Chancellor George Osborne signalled the govern...,"In a speech last month , Chancellor George Osb...",291,217,53,35


In [6]:
df.sample(5)

,src,tgt,src_char_count,tgt_char_count,src_word_count,tgt_word_count
43012,Roman civil law as codified in the Corpus Juri...,"The Corpus Juris Civilis , which codified Roma...",156,153,27,27
55836,Provided some additional requirements are met ...,As long as certain supplementary conditions ar...,54,72,10,13
60978,"Outside of kayfabe , this is done to give wres...",Wrestlers are given a break from the action ou...,194,181,40,37
153277,Different people come up with different bracke...,Various individuals create distinct brackets d...,94,77,14,11
78888,The new Kievan state prospered due to its abun...,The Kievan state flourished as a result of its...,179,182,34,35


In [7]:
pd.DataFrame(df.isna().sum(),columns=['missing_count'])

,missing_count
src,0
tgt,0
src_char_count,0
tgt_char_count,0
src_word_count,0
tgt_word_count,0


In [8]:
#https://www.analyticsvidhya.com/blog/2020/04/beginners-guide-exploratory-data-analysis-text-data/
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have","n\'t":" not","\'re":" are","\'s": " is","\'d":" would",
                     "\'ll": " will","\'t":" not","\'ve": " have","\'m":" am"}


contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

In [9]:
def clean(text):
    text = re.sub('\s*\<.*?\>\s', '', text)
    text = re.sub('\s*\{.*?\}\s', '', text)
    text = re.sub("[-+@#^/|*(){}$~<>=_%:;]","",text)
    text = text.replace("\\","")
    text = re.sub("\[","",text)
    text = re.sub("\]","",text)
    text = re.sub("\<","",text)
    text = re.sub("\>","",text)
    text = re.sub("\(","",text)
    text = re.sub("\)","",text)
    text = re.sub("[0-9]","",text)
    text = ' '.join(text.split())
    return text

In [10]:
df['src'] = df['src'].progress_apply(clean)
df['src'] = df['src'].progress_apply(expand_contractions)

  0%|          | 0/172059 [00:00<?, ?it/s]

  0%|          | 0/172059 [00:00<?, ?it/s]

In [11]:
df['tgt'] = df['tgt'].progress_apply(clean)
df['tgt'] = df['tgt'].progress_apply(expand_contractions)

  0%|          | 0/172059 [00:00<?, ?it/s]

  0%|          | 0/172059 [00:00<?, ?it/s]

In [12]:
df.sample(5)

,src,tgt,src_char_count,tgt_char_count,src_word_count,tgt_word_count
2313,Charles the Bald received the western Frankish...,"The western Frankish territories , which make ...",93,120,15,21
124337,Planned pregnancies lower the risk of potentia...,The risk of serious problems like low birth we...,137,143,21,23
121942,"Sir Michael , , is said to have tried to shelt...","It is reported that Sir Michael , aged , attem...",64,77,16,17
119562,"Hungary , Russia and Turkey had of .","None of Hungary , Russia , and Turkey had any ...",39,65,9,15
60499,The first highspeed domestic trains started in...,"In June , the initial highspeed trains for dom...",82,110,14,19


In [13]:
df.isna().sum()

src               0
tgt               0
src_char_count    0
tgt_char_count    0
src_word_count    0
tgt_word_count    0
dtype: int64

In [14]:
df.to_csv('preprocessed_csv02.csv', index=False)

In [15]:
import numpy as np
def show_random_datapoints(n_samples,df):
    for i in range(n_samples):
        id = int(np.random.uniform(0,len(df)))

        if len(df['src'].iloc[id].split())>5:
            print(f"RESPONSE: {df['src'].iloc[id]}")
            print(f"SOLUTION: {df['tgt'].iloc[id]}")
            print('*'*100)

In [16]:
show_random_datapoints(10,df)

RESPONSE: Previous accidents The crash , which took place near the Grand Canyon , was caused when the helicopter  is main rotor blade struck the wall .
SOLUTION: The collision near the Grand Canyon was due to the helicopter  is main rotor blade hitting the wall , according to past incidents .
****************************************************************************************************
RESPONSE: The Qing divided the positions into civil and military positions , each having nine grades or ranks , each subdivided into a and b categories .
SOLUTION: The Qing categorized the positions into civil and military positions , with nine grades or ranks for each , and further divided into a and b categories .
****************************************************************************************************
RESPONSE: Samskrita Bharati is an organisation working for Sanskrit revival .
SOLUTION: Samskrita Bharati is a group dedicated to the revitalization of Sanskrit .
***********************